In [19]:
import warnings
warnings.filterwarnings('ignore')
# import cv2
import numpy as np
import glob
from keras.models import load_model
import tensorflow as tf
from keras import optimizers
import keras
import segmentation_models as sm

from architecture import create_model, create_callbacks


BACKUP_MODEL_PATH = "./../Models/backup.h5"
seed = 56

IMAGE_HEIGHT = IMAGE_WIDTH = 256 * 2 # * 2

EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 4

ROOT = "./../Data/Hannover/"
TRAIN_DIR = "_Train512/"

In [20]:
model = create_model(IMAGE_HEIGHT, IMAGE_WIDTH)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 512, 512, 16  448         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_72 (BatchN  (None, 512, 512, 16  64         ['conv2d_76[0][0]']              
 ormalization)                  )                                                           

In [21]:
opt = keras.optimizers.Adam(LEARNING_RATE)
callbacks = create_callbacks(BACKUP_MODEL_PATH)

model.compile(
      optimizer=opt,
      loss=sm.losses.DiceLoss(),
      metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore()])

In [22]:
import get_data_generators

augmentation = {
"featurewise_center": "False",
"samplewise_center": "False",
"featurewise_std_normalization": "False",
"samplewise_std_normalization": "False",
# "zca_whitening": "False",
# "zca_epsilon": "False",
"rotation_range": 0,
# "width_shift_range": "False",
# "height_shift_range": "False",
# "shear_range": "False",
# "zoom_range": "False",
# "channel_shift_range": "False",
# "fill_mode"],
# "cval": "False",
"horizontal_flip": "False",
"vertical_flip": "False",
#"rescale"]),
"validation_split": 0.08
}

train_gen, val_gen = get_data_generators.getDataGenerators(augmentation, (IMAGE_WIDTH, IMAGE_HEIGHT), ROOT + TRAIN_DIR + "Images", ROOT + TRAIN_DIR + "Masks", batch_size=BATCH_SIZE, seed=seed)

Found 3607 images belonging to 1 classes.
Found 313 images belonging to 1 classes.
Found 3607 images belonging to 1 classes.
Found 313 images belonging to 1 classes.


In [ ]:
"""Load Model backup to resume training after crash or break"""

model = load_model(BACKUP_MODEL_PATH, custom_objects={"dice_loss" : sm.losses.DiceLoss(), "iou_score": sm.metrics.IOUScore(threshold=0.5), "f1-score":sm.metrics.FScore()})

In [23]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

model.fit(train_gen.data,  validation_data=val_gen.data, steps_per_epoch=train_gen.samplesPerBatch, validation_steps=val_gen.samplesPerBatch, callbacks=list(callbacks), epochs=EPOCHS, verbose=1) #,

Num GPUs Available:  1
Epoch 1/100
902/901 [==============================] - ETA: 0s - loss: 0.8690 - iou_score: 0.0799 - f1-score: 0.1310
Epoch 1: val_loss improved from inf to 0.83637, saving model to ./../Models\backup.h5
901/901 [==============================] - 141s 154ms/step - loss: 0.8690 - iou_score: 0.0799 - f1-score: 0.1310 - val_loss: 0.8364 - val_iou_score: 0.0982 - val_f1-score: 0.1642 - lr: 1.0000e-04
Epoch 2/100
902/901 [==============================] - ETA: 0s - loss: 0.7908 - iou_score: 0.1361 - f1-score: 0.2092
Epoch 2: val_loss improved from 0.83637 to 0.76546, saving model to ./../Models\backup.h5
901/901 [==============================] - 132s 147ms/step - loss: 0.7908 - iou_score: 0.1361 - f1-score: 0.2092 - val_loss: 0.7655 - val_iou_score: 0.1478 - val_f1-score: 0.2357 - lr: 1.0000e-04
Epoch 3/100
447/901 [=============>................] - ETA: 1:04 - loss: 0.7358 - iou_score: 0.1717 - f1-score: 0.2642

KeyboardInterrupt: 

In [8]:
import datetime

print("Time of saving model:", datetime.datetime.now())

model_path = "./../Models/Binary/256/bike_mapper_scratch_Train8816_Val766_Test2395_No-Aug.h5"
model.save(model_path)

2022-10-22 01:53:20.939756


In [4]:
#Source: https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2
from keras import backend as K
# def iou_coef(y_true, y_pred, smooth=1):
#   intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
#   union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
#   iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
#
#   return iou
#
# def dice_coef(y_true, y_pred, smooth = 1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
#
# def soft_dice_loss(y_true, y_pred):
#     return 1-dice_coef(y_true, y_pred)